# Ryzen AI CVML Integration with ROS

In this notebook, we'll integrate the Ryzen AI CVML library with ROS 2 to create NPU-accelerated vision nodes for robotics applications.

## Goals

* Understand the `ryzen_ai_cvml` ROS package architecture
* Launch CVML nodes with ROS 2
* Visualize depth estimation and face detection outputs in ROS
* Learn how to integrate NPU acceleration into your own ROS projects

## References

* [Writing a Simple Publisher and Subscriber (C++)](https://docs.ros.org/en/kilted/Tutorials/Beginner-Client-Libraries/Writing-A-Simple-Cpp-Publisher-And-Subscriber.html)
* [Using ROS 2 Launch Files](https://docs.ros.org/en/kilted/Tutorials/Intermediate/Launch/Launch-Main.html)
* [cv_bridge](https://github.com/ros-perception/vision_opencv/tree/ros2/cv_bridge)

## The `ryzen_ai_cvml` ROS Package

The `ryzen_ai_cvml` package wraps the CVML C++ API into ROS 2 nodes. It provides:

### Nodes
- **depth_estimation_node**: Subscribes to image topics, publishes depth maps
- **face_detection_node**: Detects faces and publishes bounding boxes + landmarks
- **face_mesh_node**: Generates 3D face meshes (468 landmarks)

## Explore the Package Structure

Let's look at what's in the `ryzen_ai_cvml` package:

In [3]:
!ls -R ryzen_ai_cvml/

ryzen_ai_cvml/:
CMakeLists.txt	include  launch  package.xml  README.md  scripts  src

ryzen_ai_cvml/include:
ryzen_ai_cvml

ryzen_ai_cvml/include/ryzen_ai_cvml:
depth_estimation_node.hpp  face_detection_node.hpp  face_mesh_node.hpp

ryzen_ai_cvml/launch:
depth_estimation.launch.py  face_detection.launch.py  face_mesh.launch.py

ryzen_ai_cvml/scripts:
video_publisher.py

ryzen_ai_cvml/src:
depth_estimation_node.cpp  face_detection_node.cpp  face_mesh_node.cpp
face_detection_main.cpp    face_mesh_main.cpp	    main.cpp


## Building the ROS Package

If the package isn't built yet, we need to build it with colcon:

In [11]:
%%bash
# This cell might take a few minutes

cd ryzen_ai_cvml

# Check if already built
if [ -d "build" ]; then
    echo "Package appears to be already built."
    echo "Skipping build. If you need to rebuild, run: colcon build --packages-select ryzen_ai_cvml"
else
    echo "Building ryzen_ai_cvml package..."
    source /opt/ros/kilted/setup.bash
    cd ..
    colcon build --packages-select ryzen_ai_cvml
    echo "Build complete!"
fi

Building ryzen_ai_cvml package...
Starting >>> ryzen_ai_cvml
Finished <<< ryzen_ai_cvml [0.40s]

Summary: 1 package finished [0.48s]
Build complete!


## Launch Loopback Video Publisher

For this part you will want to open a new jupyter terminal

![](images/new_terminal.png)

Type in the following:

```bash
source install/setup.bash

ros2 run ryzen_ai_cvml video_publisher.py \
    --ros-args \
    -p video_path:=/ryzers/RyzenAI-SW/Ryzen-AI-CVML-Library/samples/video_call.mp4 \
    -p topic:=/camera/image_raw
```

You should see these messages if successful:
```
[INFO] [1760488629.706959565] [video_publisher]: Publishing video from: /ryzers/RyzenAI-SW/Ryzen-AI-CVML-Library/samples/video_call.mp4
[INFO] [1760488629.707166260] [video_publisher]: Publishing to topic: /camera/image_raw
```

Let's see if ROS detects our new node

In [18]:
!ros2 node list

/video_publisher


Excellent! We have a video_publisher node active in our system. Now let's use the CVML depth estimation node to subscribe to it.

## Launch Depth Estimation Pipeline

For this we will want yet another new terminal

![](images/new_terminal.png)

Now type these commands into the new terminal:

```bash
source /opt/ros/kilted/setup.bash
source install/setup.bash
ros2 launch ryzen_ai_cvml depth_estimation.launch.py
```

The node will subscribe to our video_publisher and publish depth maps on `/depth/image`. Once all goes well you should observe the following:

```
[INFO] [launch]: All log files can be found below /root/.ros/log/2025-10-15-00-42-50-071596-stxh-rad-126898
[INFO] [launch]: Default logging verbosity is set to INFO
[INFO] [depth_estimation_node-1]: process started with pid [126902]
[depth_estimation_node-1] [INFO] [1760488970.197557193] [depth_estimation_node]: Input topic: /camera/image_raw
[depth_estimation_node-1] [INFO] [1760488970.197609302] [depth_estimation_node]: Output topic: /depth_estimation/depth
[depth_estimation_node-1] [INFO] time:185810749 thread:132226191791872 AMD CVML SDK: 0.0.0-dev
[depth_estimation_node-1] [INFO] time:185810749 thread:132226191791872 Any GPU inference will use AMD Radeon Graphics (RADV GFX1151)[0]
[depth_estimation_node-1] [INFO] time:185810749 thread:132226191791872 Any GPU inference will use AMD Radeon Graphics (RADV GFX1151)[0]
[depth_estimation_node-1] [INFO] time:185810758 thread:132226191791872 [Depth Estimation] Using ONNX engine, NPU backend
[depth_estimation_node-1] [INFO] [1760488970.246808904] [depth_estimation_node]: Ryzen AI Depth Estimation initialized
[depth_estimation_node-1] [INFO] [1760488970.491575724] [depth_estimation_node]: Created publisher on: /depth_estimation/depth
[depth_estimation_node-1] [INFO] [1760488970.491899477] [depth_estimation_node]: Created subscriber on: /camera/image_raw
[depth_estimation_node-1] [INFO] [1760488970.491906947] [depth_estimation_node]: Depth estimation node started successfully
[depth_estimation_node-1] [INFO] [1760488970.525778512] [depth_estimation_node]: Received first image: 1920x1080
[depth_estimation_node-1] WARNING: Logging before InitGoogleLogging() is written to STDERR
[depth_estimation_node-1] I20251015 00:42:50.540738 126927 vitisai_compile_model.cpp:1143] Vitis AI EP Load ONNX Model Success
[depth_estimation_node-1] I20251015 00:42:50.540784 126927 vitisai_compile_model.cpp:1144] Graph Input Node Name/Shape (1)
[depth_estimation_node-1] I20251015 00:42:50.540791 126927 vitisai_compile_model.cpp:1148]       efficient_Unet::input_0_nhwc : [1x256x256x3]
[depth_estimation_node-1] I20251015 00:42:50.540795 126927 vitisai_compile_model.cpp:1154] Graph Output Node Name/Shape (1)
[depth_estimation_node-1] I20251015 00:42:50.540797 126927 vitisai_compile_model.cpp:1158]       2196_nhwc : [1x256x256x1]
[depth_estimation_node-1] [Vitis AI EP] No. of Operators :   CPU     2    NPU   616 
[depth_estimation_node-1] [Vitis AI EP] No. of Subgraphs :   NPU     1 Actually running on NPU     1 
[depth_estimation_node-1] [INFO] time:185811364 thread:132224952436416 [ONNX VAI] Session created
[depth_estimation_node-1] [INFO] [1760488970.872380363] [depth_estimation_node]: Published first depth map
```

Key lines here are
```
[depth_estimation_node-1] [INFO] [1760488970.246808904] [depth_estimation_node]: Ryzen AI Depth Estimation initialized
[depth_estimation_node-1] [INFO] [1760488970.491575724] [depth_estimation_node]: Created publisher on: /depth_estimation/depth
[depth_estimation_node-1] [INFO] [1760488970.491899477] [depth_estimation_node]: Created subscriber on: /camera/image_raw
[depth_estimation_node-1] [INFO] [1760488970.491906947] [depth_estimation_node]: Depth estimation node started successfully
[depth_estimation_node-1] [INFO] [1760488970.525778512] [depth_estimation_node]: Received first image: 1920x1080
```

### Check Active Topics and Nodes

In [19]:
!ros2 node list

/depth_estimation_node
/video_publisher


Now for the topics

In [23]:
!ros2 topic list

/camera/image_raw
/depth_estimation/depth
/parameter_events
/rosout


Also, let's make sure the NPU is actually being utilized

In [22]:
!/opt/xilinx/xrt/bin/xrt-smi examine --report aie-partitions


--------------------------------
[0000:c6:00.1] : NPU Strix Halo
--------------------------------
AIE Partitions
  Total Memory Usage: N/A
  Partition Index   : 0
    Columns: [0, 1, 2, 3, 4, 5, 6, 7]
    HW Contexts:
      |PID                 |Ctx ID     |Submissions |Migrations  |Err  |Priority |
      |Process Name        |Status     |Completions |Suspensions |     |GOPS     |
      |Memory Usage        |Instr BO   |            |            |     |FPS      |
      |                    |           |            |            |     |Latency  |
      |====================|===========|============|============|=====|=========|
      |189779              |1          |8257        |0           |0    |Normal   |
      |N/A                 |Active     |8257        |37          |     |9        |
      |N/A                 |1712 KB    |            |            |     |N/A      |
      |                    |           |            |            |     |N/A      |
      |--------------------|------

Great, now we have a mp4 -> video_publisher -> depth_estimation pipeline! What are we missing? We need to visualize the output

## Visualize the Pipeline

```bash
ros2 run web_video_server web_video_server --ros-args -p port:=8080 -p address:=0.0.0.0
```

In [27]:
!ros2 topic list

/camera/image_raw
/depth_estimation/depth
/parameter_events
/rosout


In [ ]:
from IPython.display import HTML

HTML(f'''
    <img src="http://localhost:8080/stream?topic=/depth_estimation/depth&type=mjpeg" 
         width="640" height="480">
''')

We can also check out the raw video stream by replacing the topic

In [ ]:
HTML(f'''
    <img src="http://localhost:8080/stream?topic=/camera/image_raw&type=mjpeg" 
         width="640" height="480">
''')

## Key Takeaways

### Integration Pattern

1. **CVML Context** → Create and configure backend
2. **CVML Feature** → Initialize the vision feature (depth, face, etc.)
3. **ROS Subscriber** → Receive image messages
4. **cv_bridge** → Convert between ROS and OpenCV formats
5. **CVML Processing** → NPU-accelerated inference
6. **ROS Publisher** → Publish results

### Benefits of NPU in ROS

- **Power Efficient**: Lower power draw compared to GPU/CPU
- **Dedicated Hardware**: Frees up CPU/GPU for other tasks
- **Real-time Performance**: ~30 FPS for depth estimation
- **Easy Integration**: Same ROS patterns as any other vision node

### Production Considerations

1. **Backend Selection**: Use parameters to allow runtime backend switching
2. **Error Handling**: Check NPU availability, handle fallback to CPU
3. **Performance Monitoring**: Use `xrt-smi` to verify NPU utilization
4. **Resource Management**: Properly release CVML contexts on shutdown

## Next Steps

You've now learned how to:
- ✅ Validate and use the NPU with `xrt-smi`
- ✅ Run CVML samples on different backends
- ✅ Integrate CVML with ROS 2 for robotics applications

### Further Exploration

- Try different CVML features (face mesh, pose estimation)
- Create custom ROS nodes with CVML integration
- Combine multiple CVML features in a single pipeline
- Benchmark NPU vs GPU vs CPU for your specific workloads
- Build custom NPU applications with MLIR-AIE (advanced)

### Resources

- [Ryzen AI Documentation](https://ryzenai.docs.amd.com/)
- [XDNA Driver GitHub](https://github.com/amd/xdna-driver)
- [RyzenAI-SW GitHub](https://github.com/amd/RyzenAI-SW)
- [ROS 2 Documentation](https://docs.ros.org/)

---
Copyright© 2025 AMD, Inc SPDX-License-Identifier: MIT